In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
os.environ['SNORKELDB'] = 'postgres://localhost/geochron'

from snorkel import SnorkelSession
session = SnorkelSession()

In [2]:
from snorkel.parser import GDDDocPreprocessor
n_docs = 2000
doc_preprocessor = GDDDocPreprocessor('data/sentences_nlp352', max_docs=n_docs)

In [ ]:
from snorkel.parser.gdd_parser import Gdd
from snorkel.parser import CorpusParser

corpus_parser = CorpusParser(parser=Gdd())
%time corpus_parser.apply(doc_preprocessor, count=n_docs, parallelism=4)

Clearing existing...
Running UDF...


In [ ]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).c?ount())

print("Sentences:", session.query(Sentence).count())

In [ ]:
from snorkel.models import candidate_subclass

Measurement = candidate_subclass('Measurement', ['strat_name', 'value'])

In [ ]:
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import DictionaryMatch, RegexMatchSpan

ngrams         = Ngrams(n_max=7)
#person_matcher = PersonMatcher(longest_match_only=True)
strat_names = []
with open("data/strat_names") as fin:
    for line in fin:
        strat_names.append(line.strip().replace('"', ''))
print strat_names[:5]
    
stratname_matcher = DictionaryMatch(d=strat_names)
measurement_matcher = RegexMatchSpan(rgx = '\d+(?:\.\d*)?(?: ?[+ \-±/Ϯ]+ *\d*(?:\.\d*)* ?)*[MG]a')
cand_extractor = CandidateExtractor(Measurement, [ngrams, ngrams], [stratname_matcher, measurement_matcher])

In [ ]:
from snorkel.models import Document
from sqlalchemy.sql.expression import func
docs = session.query(Document).order_by("reverse(name)")
# docs = session.query(Document).order_by(Document.name).all()

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
        if i % 10 == 9:
            dev_sents.add(s)
        elif i % 10 == 8:
            test_sents.add(s)
        else:
            train_sents.add(s)

In [ ]:
%%time
for i, sents in enumerate([train_sents, dev_sents, test_sents]):
#for i, sents in enumerate([train_sents]):
    cand_extractor.apply(sents, split=i, parallelism=4)
    print("Number of candidates:", session.query(Measurement).filter(Measurement.split == i).count())

In [ ]:
test_cands = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()

In [ ]:
from snorkel.learning.disc_models.rnn import reRNN
lstm = reRNN(seed=1701, n_threads=None)
lstm.load("measurements.lstm")

In [ ]:
marginals = lstm.marginals(test_cands)